**[Lab 1](https://pages.github.umn.edu/deliu/bigdata19/10-SparkIntro/sparklab01/sparklab01-solution.html)**

**[Lab 2 - RDD](https://pages.github.umn.edu/deliu/bigdata19/10-SparkIntro/sparklab02-RDD-transform/sparklab2-solution.html)**

**[Lab 3 - Processing](https://pages.github.umn.edu/deliu/bigdata19/10-SparkIntro/sparklab03-RDD-ProcessFiles/sparklab3-solution.html)**

**[Lab 4 - Pairs](https://pages.github.umn.edu/deliu/bigdata19/13-PairRDD/sparklab04-PairRDD-Join/SparkLab4-Solution.html)**

In [ ]:
! wget http://idsdl.csom.umn.edu/c/share/sparkdata.zip
! unzip sparkdata.zip

!cd training_materials/data/weblogs
!ls
!cat * | head
!head -2 loudacre/webpage.json    # devices
!head $DEV1DATA/devicestatus.txt


# Save to HDFS
ips.saveAsTextFile("iplist")
!ls iplist

In [ ]:
# Read in
logfile="training_materials/data/weblogs"
logfile='file:/home/cloudera/training_materials/data/devicestatus.txt'
logs = sc.textFile(logfile).cache()


logs.count()        # Count lines
logs.collect()      # View whole dataset
logs.take(5)


# Determine delimiter
mydata.map(lambda line:line[19]).distinct().collect()


# Filter rows
parsed  = mydata.map(lambda line:line.split(line[19]))
filtered = parsed.filter(lambda record:len(record)==14)


# Get fields
selectedFields = filtered.map(lambda r:(r[0],r[1],r[2],r[12],r[13]))
for row in selectedFields.take(3):
    print("{}, {}, {}, {}, {}".format(*row))


# Split a field up
splitted = selectedFields.map(lambda r:(r[1].split(' ')[0],
                                 r[1].split(' ')[1]))

In [ ]:
# Count lines with jpg
sc.textFile(logfile) \
    .filter(lambda line:".jpg" in line) \
    .count()


# Define new rdd - array of words per line
words = logs.map(lambda line:line.split()).take(5)
for word in words.take(5):
    print(word)


# First word per line
logs.map(lambda line: line.split()[0]).take(5)


# Words per user
userCount = logs.map(lambda row:(row[1], 1))
    .reduceByKey(lambda a,b:a+b)
type(userCount)


# Histogram
freqCount = userCount.map(lambda (user,cnt):(cnt,user)) # reverse order
    .countByKey()                                       # counts per person

    
# Value_counts
for item in freqCount.items():
    print("{}:{}".format(*item))
    
    
# Count users per html
ipusers = logs.filter(lambda line:".html" in line). \
    map(lambda line: (line.split()[0], line.split()[2]))


# List of addresses per user
userIpList = logs.map(lambda row:(row[1],row[0])) \
    .groupByKey() \
    .mapValues(list)

for item in userIpList.take(5):
    print("{}\t{}".format(item[0],item[1]))